<a href="https://colab.research.google.com/github/ooutaksaluk/MBT-analysis/blob/main/Main_Battle_Tank_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Main Battle Tank analysis**



In [ ]:
!pip install anvil-uplink opencv-python numpy torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

**เตรียมข้อมูลและการโหลดข้อมูลด้วย Keras**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

train_dir = '/content/drive/My Drive/dataset/train'
validation_dir = '/content/drive/My Drive/dataset/validation'

# ตรวจสอบว่าโฟลเดอร์มีอยู่จริง
if os.path.exists(train_dir):
    print("พบโฟลเดอร์:", train_dir)
else:
    print("ไม่พบโฟลเดอร์:", train_dir)


พบโฟลเดอร์: /content/drive/My Drive/dataset/train


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # ใช้ 20% เป็น validation
)

# โหลดข้อมูล Train (ใช้ 80%)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# โหลดข้อมูล Validation (ใช้ 20%)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ADMIN\\Documents\\dataset\\train'

**Training Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# สร้างโมเดล CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # binary classification (Sight vs Mantlet)

# คอมไพล์โมเดล
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# ฝึกโมเดล
model.fit(train_generator, epochs=10, validation_data=validation_generator)





**Prediction**



In [ ]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image

# โหลดโมเดลที่ฝึกเสร็จแล้ว
model = tf.keras.models.load_model('path_to_trained_model.h5')

def predict_sight_mantlet(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (150, 150))  # ปรับขนาดให้เหมาะสมกับ input ของโมเดล
    img_array = image.img_to_array(img)  # แปลงภาพเป็น array
    img_array = np.expand_dims(img_array, axis=0)  # เพิ่ม batch dimension

    # ทำนายผล
    prediction = model.predict(img_array)

    # ตรวจสอบผลลัพธ์
    if prediction[0] > 0.5:
        return "Mantlet"
    else:
        return "Sight"

# ทดสอบโมเดล
result = predict_sight_mantlet('path_to_test_image.jpg')
print(result)


In [ ]:
import anvil.server
import numpy as np
import cv2
import io
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# เชื่อมต่อกับ Anvil
anvil.server.connect("server_LX2D76OV3RCR3XCSOMCFYQZL-JGQCERXRR5DWB5QH")

# โหลดโมเดลที่ฝึกเสร็จแล้ว
model = tf.keras.models.load_model('path_to_trained_model.h5')

@anvil.server.callable
def process_image(image_bytes):
    # แปลง bytes เป็นภาพ
    img_array = np.frombuffer(image_bytes, np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    # ปรับขนาดภาพให้เหมาะสม
    img_resized = cv2.resize(img, (150, 150))  # ปรับขนาดให้เหมาะสมกับ input ของโมเดล
    img_array = image.img_to_array(img_resized)
    img_array = np.expand_dims(img_array, axis=0)

    # ทำนายผล
    prediction = model.predict(img_array)

    # ตรวจสอบผลลัพธ์
    result = "Mantlet" if prediction[0] > 0.5 else "Sight"

    # ใส่ Label และส่งผลลัพธ์กลับ
    cv2.putText(img, result, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # แปลงกลับเป็น bytes
    _, buffer = cv2.imencode('.png', img)
    return io.BytesIO(buffer).getvalue()

# รอรับคำสั่งจาก Anvil
anvil.server.wait_forever()
